In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns
%matplotlib inline

In [5]:
df = pd.read_excel("Incidents.xlsx")
df.drop(columns=["am_ttr"], inplace=True)

In [7]:
df.describe()

,global_ttr_seconds,assignee_employee_number,csat_score,inc_ka_flag,reassignment_count,reopened_flag
count,2.184500e+04,2.184500e+04,1819.000000,21845.000000,21845.000000,21845.000000
mean,2.916809e+05,6.441109e+08,4.487631,0.507485,0.430716,0.023941
std,5.075492e+05,1.033755e+07,1.045834,0.499955,0.785978,0.152870
min,0.000000e+00,4.901220e+07,1.000000,0.000000,0.000000,0.000000
25%,7.978700e+04,6.430265e+08,4.000000,0.000000,0.000000,0.000000
50%,1.789600e+05,6.430342e+08,5.000000,1.000000,0.000000,0.000000
75%,3.452950e+05,6.430383e+08,5.000000,1.000000,1.000000,0.000000
max,1.287599e+07,1.111111e+09,5.000000,1.000000,16.000000,1.000000


In [8]:
def clean_data(df):
    '''
    INPUT
    df - pandas dataframe 
    
    OUTPUT
    X - A matrix holding all of the variables you want to consider when predicting the response
    y - the corresponding response vector
    
    This function cleans df using the following steps to produce X and y:
    1. Drop all the rows with no salaries
    2. Create X as all the columns that are not the Salary column
    3. Create y as the Salary column
    4. Drop the Salary, Respondent, and the ExpectedSalary columns from X
    5. For each numeric variable in X, fill the column with the mean value of the column.
    6. Create dummy columns for all the categorical variables in X, drop the original columns
    '''
    # Predict global_ttr_seconds = seconds from ticket creation until resolution
    y = df['global_ttr_seconds']
   
   
    # Fill numeric columns with the mean (limited to csat_score)
    num_vars = df.select_dtypes(include=['float', 'int']).columns
    for col in num_vars:
        df[col].fillna((df[col].mean()), inplace=True)
        
    # Dummy the categorical variables
    cat_vars = df.select_dtypes(include=['object']).copy().columns
    for var in  cat_vars:
        # for each cat add dummy var, drop original column
        df = pd.concat([df.drop(var, axis=1), pd.get_dummies(df[var], prefix=var, prefix_sep='_', drop_first=True)], axis=1)

    X = df

    return X, y
    
#Use the function to create X and y
X, y = clean_data(df)    

In [11]:
#Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lm_model = LinearRegression(normalize=True) # Instantiate
lm_model.fit(X_train, y_train) #Fit

#Predict using your model
y_test_preds = lm_model.predict(X_test)
y_train_preds = lm_model.predict(X_train)

#Score using your model
test_score = r2_score(y_test, y_test_preds)
train_score = r2_score(y_train, y_train_preds)


In [14]:
print("The rsquared on the training data was {}.  The rsquared on the test data was {}.".format(train_score, test_score))

The rsquared on the training data was 1.0.  The rsquared on the test data was 0.9991192651448522.


In [18]:
def coef_weights(coefficients, X_train):
    '''
    INPUT:
    coefficients - the coefficients of the linear model 
    X_train - the training data, so the column names can be used
    OUTPUT:
    coefs_df - a dataframe holding the coefficient, estimate, and abs(estimate)
    
    Provides a dataframe that can be used to understand the most influential coefficients
    in a linear model by providing the coefficient estimates along with the name of the 
    variable attached to the coefficient.
    '''
    coefs_df = pd.DataFrame()
    coefs_df['est_int'] = X_train.columns
    coefs_df['coefs'] = lm_model.coef_
    coefs_df['abs_coefs'] = np.abs(lm_model.coef_)
    coefs_df = coefs_df.sort_values('abs_coefs', ascending=False)
    return coefs_df

#Use the function
coef_df = coef_weights(lm_model.coef_, X_train)

#A quick look at the top results
coef_df.shape

(3692, 3)

In [19]:
coef_df.to_excel("IncidentDuration_coef.xlsx")